In [1]:
!pip install roboflow -q
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 25.0 MB/s eta 0:00:00


In [2]:
from roboflow import Roboflow
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Roboflow 데이터 불러오기

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to glassbottle-6 in yolov11:: 100%|██████████| 198/198 [00:00<00:00, 1471.17it/s]


#### 1) Label 파일의 클래스 값 변경

In [ ]:
import os
import shutil

In [ ]:
def modify_class_2(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    lines = file.readlines()

                # 모든 줄의 첫 번째 값이 1인 경우 2로 변경
                for i in range(len(lines)):
                    parts = lines[i].strip().split()
                    if parts and parts[0] == "1":
                        parts[0] = "2"  # 첫 번째 값이 1인 경우 2로 변경
                        lines[i] = " ".join(parts) + "\n"

                # 변경된 내용을 파일에 다시 저장
                with open(file_path, "w") as file:
                    file.writelines(lines)

modify_class_2('/content/crime-Detection-5')

In [ ]:
def modify_class_1(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    lines = file.readlines()

                # 모든 줄의 첫 번째 값을 1로 변경
                for i in range(len(lines)):
                    parts = lines[i].strip().split()
                    if parts:
                        parts[0] = "1"  # 첫 번째 값을 1로 변경
                        lines[i] = " ".join(parts) + "\n"

                # 변경된 내용을 파일에 다시 저장
                with open(file_path, "w") as file:
                    file.writelines(lines)

modify_class_1('/content/crimeDetection-7')

In [ ]:
def modify_class_0(folder_path):
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    lines = file.readlines()

                # 모든 줄의 첫 번째 값을 0으로 변경
                for i in range(len(lines)):
                    parts = lines[i].strip().split()
                    if parts:
                        parts[0] = "0"  # 첫 번째 값을 0으로 변경
                        lines[i] = " ".join(parts) + "\n"

                # 변경된 내용을 파일에 다시 저장
                with open(file_path, "w") as file:
                    file.writelines(lines)

modify_class_0('/content/crime-1')

#### 2) 모델이 요하는 구조의 폴더 생성

In [ ]:
## 생성될 폴더의 경로
crime_detect_data = '/content/CrimeDetect-2'

## 폴더가 존재하지 않을 때 폴더를 생성
if not os.path.exists(crime_detect_data) :
    os.makedirs(crime_detect_data)

## 폴더 생성 확인
print(os.path.exists(crime_detect_data))

NameError: name 'os' is not defined

In [ ]:
## 생성될 폴더에 대한 하위 폴더 생성
data_split = ['train', 'test', 'valid']

for ds in data_split :
    temp = os.path.join(crime_detect_data, ds)

    if not os.path.exists( temp ) :
        os.makedirs(temp)

    ## 폴더 생성 확인
    print(os.path.exists(temp))

True
True
True


In [ ]:
## 생성될 폴더에 대한 하위 폴더 생성
image_label = ['images', 'labels']

for ds in data_split:
    for il in image_label:
        temp = os.path.join(crime_detect_data, ds, il)

        if not os.path.exists(temp):
            os.makedirs(temp)

        # 폴더 생성 확인
        print(os.path.exists(temp))

True
True
True
True
True
True


#### 3) 각 폴더에 데이터 옮기기

In [ ]:
def copy_data(source_path, destination_path):
    # 필요한 상위 폴더 및 하위 폴더 리스트
    data_splits = ['train', 'test', 'valid']
    sub_folders = ['images', 'labels']

    # 각 폴더 복사
    for split in data_splits:
        for sub_folder in sub_folders:
            src_dir = os.path.join(source_path, split, sub_folder)
            dest_dir = os.path.join(destination_path, split, sub_folder)

            # 폴더가 없으면 생성
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)

            # 파일 복사
            if os.path.exists(src_dir):
                for file_name in os.listdir(src_dir):
                    src_file = os.path.join(src_dir, file_name)
                    dest_file = os.path.join(dest_dir, file_name)

                    if os.path.isfile(src_file):  # 파일만 복사
                        shutil.copy2(src_file, dest_file)
    print("복사 완료")

In [ ]:
source_path = ['/content/crime-Detection-5', '/content/crimeDetection-7', '/content/crime-1']
destination_path = '/content/crime_detect'

for sp in source_path:
    copy_data(sp, destination_path)

복사 완료
복사 완료
복사 완료


In [ ]:
def zip_folder(folder_path, output_path):
    # 압축 파일 이름과 경로를 지정합니다.
    archive_name = os.path.basename(folder_path)
    shutil.make_archive(output_path + archive_name, 'zip', folder_path)
    print(f"{archive_name}.zip 파일이 {output_path}에 생성되었습니다.")

# 사용 예시
folder_path = '/content/crime_detect'  # 압축할 폴더 경로
output_path = '/content/' # 생성될 ZIP 파일 경로
zip_folder(folder_path, output_path)

fire_detect.zip 파일이 /content/에 생성되었습니다.


In [ ]:
yaml_data = {
    'train': '../train/images',
    'val': '../valid/images',
    'test': '../test/images',
    'nc': 3,
    'names': ['High', 'Middle', 'Low']
}

In [ ]:
import yaml

In [ ]:
with open("/content/crime_detect/data.yaml", "w") as f:
    yaml.dump(yaml_data, f)

#### 4) 모델 학습

In [4]:
# 모델 선언
model = YOLO('yolo11m.pt')

100%|██████████| 38.8M/38.8M [00:00<00:00, 135MB/s]


In [5]:
# 모델 학습
model.train(data='/content/glassbottle-6/data.yaml', batch=-1, epochs=100, imgsz=640, save=True)

Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/glassbottle-6/data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 755k/755k [00:00<00:00, 25.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 87.7MB/s]


AMP: checks passed ✅


...

                   all          7          7      0.849          1       0.96      0.583



100 epochs completed in 0.129 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  7.94it/s]


                   all          7          7          1      0.978      0.995      0.726
Speed: 0.2ms preprocess, 10.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77faf0526ec0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
# 모델 저장
model.save('/content/drive/MyDrive/AIVLE/Project/BigProject/CrimeDetect/Model/YOLO11m-bottle.pt')